# Import Libraries

In [1]:
import cv2
import mediapipe as mp

# Setup Mediapipe

In [8]:
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    enable_segmentation=False,
    smooth_segmentation=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

UPPER_BODY_LANDMARKS = {
    11, 12, 13, 14, 15, 16
}
UPPER_BODY_CONNECTIONS = [
    (11, 13), 
    (13, 15),  
    (12, 14),
    (14, 16),
    (11, 12)   
]

# Hand and Upper Body Detection

In [ ]:
def hand_and_upper_body_detect():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        results_hands = hands.process(frame_rgb)
        
        results_pose = pose.process(frame_rgb)

        if results_hands.multi_hand_landmarks:
            for hand_landmarks in results_hands.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                h, w, c = frame.shape
            

        if results_pose.pose_landmarks:
            h, w, c = frame.shape
            landmarks = results_pose.pose_landmarks.landmark
            
            for idx in UPPER_BODY_LANDMARKS:
                lm = landmarks[idx]
                cx, cy = int(lm.x * w), int(lm.y * h)
                cv2.circle(frame, (cx, cy), 5, (255, 0, 0), -1) 
            
            for conn in UPPER_BODY_CONNECTIONS:
                start_idx, end_idx = conn
                start_lm = landmarks[start_idx]
                end_lm = landmarks[end_idx]
                start_point = (int(start_lm.x * w), int(start_lm.y * h))
                end_point = (int(end_lm.x * w), int(end_lm.y * h))
                cv2.line(frame, start_point, end_point, (255, 0, 0), 2)
            

        cv2.imshow("Hand and Upper Body Detection", frame)

        if cv2.waitKey(1) & 0xFF == 27: 
            break

    cap.release()
    cv2.destroyAllWindows()

In [12]:
hand_and_upper_body_detect()